In [1]:
import recpack

In [2]:
import pandas as pd

In [3]:
dataframe = pd.read_csv(
    'gs://froomle-playground-robin-1-data/datasets/ML_100K/u.data',
    sep='\t',
    header=None,
    names = ['userId', 'movieId', 'rating', 'timestamp']
)

/Users/robinverachtert/workspace/doctorate/dist-env/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
from recpack import algorithms

In [5]:
from recpack import helpers

In [6]:
from recpack import evaluate

In [7]:
item_ids = list(dataframe['movieId'].unique())
user_ids = list(dataframe['userId'].unique())

In [8]:
item_id_mapping = helpers.rescale_id_space(item_ids)

In [9]:
user_id_mapping = helpers.rescale_id_space(user_ids)

In [10]:
dataframe['iid'] = dataframe['movieId'].map(lambda x: item_id_mapping[x])
dataframe['uid'] = dataframe['userId'].map(lambda x: user_id_mapping[x])

In [11]:
df = dataframe.drop(['userId', 'movieId', 'rating', 'timestamp'], axis=1)

In [12]:
# Still need to move over the train test split function. 

In [13]:
sp_mat = helpers.create_csr_matrix_from_pandas_df(df, 'iid', 'uid')

In [23]:
algo = algorithms.get_algorithm('ease')

In [24]:
A = algo()

In [25]:
A.fit(sp_mat)

array([[ 0.00000000e+00,  1.25653464e-02, -3.49344744e-04, ...,
         1.99267437e-04,  1.99267437e-04,  1.99267437e-04],
       [ 1.18417712e-02,  0.00000000e+00,  1.17905741e-03, ...,
        -2.49942997e-04, -2.49942997e-04, -2.49942997e-04],
       [-3.76554206e-04,  1.34854671e-03,  0.00000000e+00, ...,
        -6.68682987e-05, -6.68682987e-05, -6.68682987e-05],
       ...,
       [ 2.16744564e-04, -2.88476615e-04, -6.74774856e-05, ...,
         0.00000000e+00,  7.43064622e-04,  7.43064622e-04],
       [ 2.16744564e-04, -2.88476615e-04, -6.74774856e-05, ...,
         7.43064622e-04,  0.00000000e+00,  7.43064622e-04],
       [ 2.16744564e-04, -2.88476615e-04, -6.74774856e-05, ...,
         7.43064622e-04,  7.43064622e-04,  0.00000000e+00]])

In [17]:
import os

In [18]:
os.getcwd()

'/Users/robinverachtert/workspace/doctorate/recpack/notebooks'

In [19]:
# A.save(f"{os.getcwd()}/B")

In [20]:
A.load(f"{os.getcwd()}/B.npy")

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.73249368e-03, ...,
        1.80252013e-04, 1.80252013e-04, 1.80252013e-04],
       [0.00000000e+00, 2.91595492e-03, 0.00000000e+00, ...,
        8.58224121e-06, 8.58224121e-06, 8.58224121e-06],
       ...,
       [0.00000000e+00, 1.98520080e-04, 8.85734153e-06, ...,
        0.00000000e+00, 5.35271072e-04, 5.35271072e-04],
       [0.00000000e+00, 1.98520080e-04, 8.85734153e-06, ...,
        5.35271072e-04, 0.00000000e+00, 5.35271072e-04],
       [0.00000000e+00, 1.98520080e-04, 8.85734153e-06, ...,
        5.35271072e-04, 5.35271072e-04, 0.00000000e+00]])

In [26]:
metric1 = evaluate.NDCGK(20)

In [27]:
metric2 = evaluate.RecallK(20)

In [28]:
evaluator = evaluate.FoldInPercentage(0.8, sp_mat)

In [29]:
for _in, _out in evaluator:
    
    if _in.nnz == 0 or _out.nnz == 0:
        print("oops")
        continue
        
    
    
    X_pred = A.predict(_in)
    
    metric1.update(X_pred[0], _out)
    metric2.update(X_pred[0], _out)


In [30]:
metric1.value

0.2648818031679217

In [31]:
metric2.value

0.29645856570926093